Импортируйте библиотеку pandas как pd. Прочитайте датасет bookings.csv, находящийся по ссылке, с разделителем ;  и сохраните в переменную bookings. Затем запишите первые 7 строк в переменную bookings_head

In [9]:
import pandas as pd
from datetime import datetime
bookings = pd.read_csv('https://stepik.org/media/attachments/lesson/360344/bookings.csv', encoding='Windows-1251', sep=';')

In [36]:
bookings_head = bookings[:7] 

In [40]:
bookings_head.shape

(7, 21)

In [44]:
bookings.dtypes.value_counts()

object     10
int64      10
float64     1
Name: count, dtype: int64

Как вы уже могли заметить, при сборе датасета произошла небольшая ошибка, и названия колонок были записаны в неудобном для дальнейшей работы формате. Исправьте ситуацию: приведите названия столбцов к нижнему регистру и замените пробелы на знак нижнего подчёркивания. Например: Is Canceled -> is_canceled

In [18]:
bookings.columns = bookings.columns.str.lower().str.replace(' ', '_')
bookings.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

Перейдём к исследованию данных! Пользователи из каких стран совершили наибольшее число успешных бронирований? 
Бронирование считается успешным, если в дальнейшем оно не было отменено (переменная is_canceled). В качестве ответа выберите страны, входящие в топ-5.

In [231]:
(bookings.query("is_canceled == 0")
    .groupby('country', as_index = False)
    .size() #считает количество строк в каждой группе!
    .rename(columns={'size': 'successful_bookings'}) #size по умолчанию возвращает series, а не dataframe. Нужно переименовывать
    .sort_values(by='successful_bookings', ascending = False) #финальная сортировка по количеству успешных букингов
    .head(5))

,country,successful_bookings
125,PRT,21071
57,GBR,9676
54,FRA,8481
50,ESP,6391
42,DEU,6069


На сколько ночей (stays_total_nights)  в среднем бронируют отели типа City Hotel? Resort Hotel? Запишите полученные значения в пропуски с точностью до 2 знаков после точки.



In [249]:
(bookings.groupby('hotel', as_index = False)
 .aggregate({'stays_total_nights': 'mean'})
 .round(2)
)

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


Иногда тип номера, полученного клиентом (assigned_room_type), отличается от изначально забронированного (reserved_room_type). Такое может произойти, например, по причине овербукинга.

Сколько подобных наблюдений встретилось в датасете?

*отмена бронирования также считается

In [28]:
bookings.query('assigned_room_type != reserved_room_type') #Сначала думал, что надо учитывать и несовпадение, и отмены (потом или). Оказалось нет, последнее условие скорее намёк на то что не нужно доп условий. Ну ок)

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
12,Resort Hotel,0,68,2015-07-01,2015,July,27,1,0,4,...,2,0.0,0,BB,USA,D,E,Transient,Check-Out,2015-07-05
15,Resort Hotel,0,68,2015-07-01,2015,July,27,1,0,4,...,2,0.0,0,BB,IRL,D,E,Transient,Check-Out,2015-07-05
17,Resort Hotel,0,12,2015-07-01,2015,July,27,1,0,1,...,2,0.0,0,BB,IRL,A,E,Transient,Check-Out,2015-07-02
18,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,1,...,2,0.0,0,BB,FRA,A,G,Transient,Check-Out,2015-07-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119273,City Hotel,0,213,2017-08-28,2017,August,35,28,1,3,...,1,0.0,0,HB,PRT,A,K,Transient-Party,Check-Out,2017-09-01
119274,City Hotel,0,213,2017-08-28,2017,August,35,28,1,3,...,1,0.0,0,HB,PRT,A,K,Transient-Party,Check-Out,2017-09-01
119289,City Hotel,0,25,2017-08-30,2017,August,35,30,0,3,...,3,0.0,0,BB,ITA,E,F,Transient,Check-Out,2017-09-02
119297,City Hotel,0,332,2017-08-31,2017,August,35,31,0,2,...,2,0.0,0,BB,GBR,D,F,Transient,Check-Out,2017-09-02


Теперь проанализируйте даты запланированного прибытия (arrival_date_year).

На какой месяц чаще всего оформляли бронь в 2016 году? Изменился ли самый популярный месяц в 2017 году?

In [77]:
(bookings
    .query('arrival_date_year == 2017')
    .groupby('arrival_date_month', as_index = False)
    .size() #считает количество строк в каждой группе!
    .rename(columns={'size': 'booking_quantity'}) #size по умолчанию возвращает series, а не dataframe. Нужно переименовывать
    .sort_values(by='booking_quantity', ascending = False)
)

,arrival_date_month,booking_quantity


 Сгруппируйте данные по годам, а затем проверьте, на какой месяц (arrival_date_month) бронирования отеля типа City Hotel отменялись чаще всего в 2015? 2016? 2017? 

Hint: для этого сначала примените метод groupby, после чего выберите нужную колонку для дальнейших действий :) Например:

df.groupby('column_1')['column_2'].what_to_do()

In [96]:
bookings.query('hotel == "City Hotel" and is_canceled == 1') \
    .groupby('arrival_date_year')['arrival_date_month'] \
    .value_counts() \
    .groupby('arrival_date_year') \
    .idxmax()

arrival_date_year
2015    (2015, September)
2016      (2016, October)
2017          (2017, May)
Name: count, dtype: object

Посмотрите на числовые характеристики трёх колонок: adults, children и babies. Какая из них имеет наибольшее среднее значение?

In [111]:
bookings[['adults', 'children', 'babies']].mean().idxmax()

'adults'

Создайте колонку total_kids, объединив столбцы children и babies. Для отелей какого типа среднее значение переменной оказалось наибольшим?

City hotel – отель находится в городе
Resort hotel – отель курортный
В качестве ответа укажите наибольшее среднее total_kids, округлив до 2 знаков после точки.

In [269]:
bookings['total_kids'] = bookings['babies'] + bookings['children']
(bookings.groupby('hotel', as_index = False)
 .aggregate({'total_kids': 'mean'})
 .round(2)
)

,hotel,total_kids
0,City Hotel,0.10
1,Resort Hotel,0.14


In [320]:
bookings['has_kids'] = bookings['babies'] + bookings['children'] >= 1


In [ ]:
bookings['has_kids'] = bookings['total_kids'] >= 1

In [322]:
bookings.query('total_kids == has_kids')

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date,total_kids,has_kids
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0,False
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01,0.0,False
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02,0.0,False
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02,0.0,False
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017-08-30,2017,August,35,30,2,5,...,0,BB,BEL,A,A,Transient,Check-Out,2017-09-06,0.0,False
119386,City Hotel,0,102,2017-08-31,2017,August,35,31,2,5,...,0,BB,FRA,E,E,Transient,Check-Out,2017-09-07,0.0,False
119387,City Hotel,0,34,2017-08-31,2017,August,35,31,2,5,...,0,BB,DEU,D,D,Transient,Check-Out,2017-09-07,0.0,False
119388,City Hotel,0,109,2017-08-31,2017,August,35,31,2,5,...,0,BB,GBR,A,A,Transient,Check-Out,2017-09-07,0.0,False


In [375]:
bookings['has_kids'] = bookings['total_kids'] >= 1
(bookings[['has_kids', 'is_canceled']].value_counts() / bookings['has_kids'].value_counts() * 100).round(2)

has_kids  is_canceled
False     0              62.78
          1              37.22
True      0              65.08
          1              34.92
Name: count, dtype: float64

In [346]:
bookings['%'] = 100 * bookings['is_canceled'] / bookings.groupby('has_kids')['is_canceled'].transform('sum')


In [357]:
bookings['has_kids'] = bookings['total_kids'] >= 1
churn_rate = bookings.groupby('has_kids')['is_canceled'].mean() * 100
churn_rate.round(2)

has_kids
False    37.22
True     34.92
Name: is_canceled, dtype: float64